## Data Cleaning

Let's load the some data for TSLA. Unfortunately, this data is not quite as *clean* as our NVDA data, so we'll need to do some data wrangling. The file we're looking to load is `TSLA_2015_2024.csv`.

In [ ]:
# New imports for a new notebook!
import pandas as pd

In [ ]:
# Loading the data, setting the index, and having a look
df = pd.read_csv("data/TSLA_2015_2024.csv")
df["Date"] = pd.to_datetime(df["Date"], dayfirst=True)
df.set_index("Date", inplace=True)
df

Can you see what we mean by messy? How many issues can you spot?

- Dates out of order
- Duplicate rows
- Missing values

## Ordering and Duplicates

First let's start with sorting the index.

In [ ]:
# Checking for ascending order in the index
df.index.is_monotonic_increasing

# Sorting the index in place
df.sort_index(inplace=True)

# Checking for ascending order in the index
df.index.is_monotonic_increasing

Now let's focus on duplicates:

In [ ]:
# Counting duplicates
df.duplicated().sum()

# Getting rid of duplicates
df.drop_duplicates()

# But make sure we update the variable to save our work!
df.drop_duplicates(inplace=True)

# Counting duplicates
df.duplicated().sum()

df

#### Tip: Method Chaining

**Method chaining** is a popular feature of pandas. It allows us to *chain* together several operations in a single line of code. For example, we can set the index, sort the data frame and drop any duplicates all at once. Notice we don't use `inplace` but rather re-assign to the original `df` variable.

```python
df = df.set_index("Date").sort_index().drop_duplicates()
```

## Not a Number (NaN)

### Exercise: Some Null Chain

Let's look at the missing or `NaN` values next. Previously, we saw that `info()` gave us some insight into how many missing values we had, but we can also use `isnull()`.

Can you chain `isnull()` with `sum()` to get a single value stating the total number of missing values in the data frame?

In [ ]:
## YOUR CODE GOES HERE

We can find out which rows have missing data using `isnull()`, `any()` along rows and some smart *masking*.

In [ ]:
df[df.isnull().any(axis=1)]

Now that we've identified our missing values, the big question is how to handle them. There are many approaches to this that will vary depending on the data and the further analysis you plan to carry out.

In [ ]:
# We can just drop any row that contains NaN in any column
df.dropna()

# Drop a row which contains NaN in a specific column
df.dropna(subset="Close")

# We can fill in NaNs with the average of a column
df['Volume'].fillna(df['Volume'].mean())

# We can interpolate (point on a line connecting the value of the days before and after)
df["Close"].interpolate(method="linear")

# We can forward fill, and use the value from the day before
df["Close"].ffill()

### Exercise: Cleaning up

Notice how above we didn't actually update the `df` variable, so our DataFrame is still full of missing values. Fix all missing values applying the following rules:
- Fill missing Close by linear interpolation
- Fill missing Volume with the value from the day before
- Fill missing Open with the median Open
- Fill missing High with the Close or Open, whichever is higher
- Fill missing Low with a value 3% lower than the High


Your DataFrame `df` should have no missing values when done. Use `info()` to confirm.

**NOTE:** When changing values in a data frame, it is recommended to avoid using `inplace`, and instead re-assign the variable.

In [ ]:
## YOUR CODE GOES HERE

#### Advanced: Data Types

You may have noticed that the **Volume** column in the 2021 data frame is a `float64` instead of the `int64` *dtype* we had in the 2020 data frame. Missing values (NaN) are represented as a special case of floating point number, so all the values in **Volume** were automatically *upcast* to floats.

Ideally our columns should be of the *dtype* that most accurately represents them. This will improve performance when working with large data frames. Now that we've resolved our missing numbers, we can *cast* our trading volumes as integers.

In [ ]:
df["Volume"] = df["Volume"].astype("int64")
df.info()

## Saving Data

Now that we've cleaned our data, let's save it, by writing it to a new .CSV file. We can use pandas' `to_csv()`.

In [ ]:
df.to_csv("TSLA_10_clean.csv")